In [0]:
import numpy as np

import tensorflow as tf

import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm


import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

In [0]:
import os
os.getcwd()
from google.colab import drive
import os
drive.mount('/content/gdrive')

root_path = 'gdrive/My Drive/Colab Notebooks/eva_research_team4/src/' #change dir to your project folder

os.chdir('/content/' + root_path)
os.getcwd()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'/content/gdrive/My Drive/Colab Notebooks/eva_research_team4/src'

In [0]:
#from all_models import DavidNet

import run_util


from importlib import reload
reload(run_util)

from run_util import Run

import data_pipeline

reload(data_pipeline)

from zeedensenet import ZeeDenseNet

Using TensorFlow backend.



32 conv0.9234744798103197 (3, 3) (1, 1) same (1, 1) None glorot_uniform
32 conv0.9234744798103197 (3, 3) (1, 1) same (1, 1) None glorot_uniform
32 conv0.9234744798103197 (3, 3) (1, 1) same (1, 1) None glorot_uniform


In [0]:
data_pipeline.get_data(dataset_name = "CIFAR10", tfrecords_flag=True)

saving to tf records
Instructions for updating:
Please write your own downloading logic.
Generating ../data/train/train.tfrecords


Generating ../data/eval/eval.tfrecords
getting tf records complete


######

In [0]:
obj = Run()

In [0]:
train_mean = np.array([125.30691805, 122.95039414, 113.86538318])

train_std= np.array([62.99321928, 62.08870764, 66.70489964])

normalize = lambda x: ((x - train_mean) / train_std)

def data_aug_train(x, y):
    
    #x = tf.image.per_image_standardization(x)
    
    x = normalize(x)
        
    #x = tf.image.random_flip_left_right(x)
    
    paddings = [(4, 4), (4, 4), (0, 0)]
    
    x = tf.pad(x, paddings, "REFLECT")
    
    x = tf.random_crop(x, [32, 32, 3])
    
    return (x, y)
  
def data_aug_test(x, y):
    
    #x = tf.image.per_image_standardization(x)
    
    x = normalize(x)
        
    #x = tf.image.random_flip_left_right(x)
    
    #paddings = [(4, 4), (4, 4), (0, 0)]
    
    #x = tf.pad(x, paddings, "REFLECT")
    
    #x = tf.random_crop(x, [32, 32, 3])
    
    return (x, y)

# Train

In [0]:
BATCH_SIZE = 128 #@param {type:"integer"}
MOMENTUM = 0.95 #@param {type:"number"}

MIN_MOMENTUM = 0.8 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 30 #@param {type:"integer"}


MIN_LEARNING_RATE = 0.000001 #@param {type:"number"}

END_LR_SMOOTHING_PERC = 0.15 #@param {type:"number"}

COMMENTS = "Zeedense 7M without roots" #@param {type:"string"}


params_tune = {
    
  "epochs":EPOCHS, 
 
  "batch_size" : BATCH_SIZE,

  "max_lr": LEARNING_RATE,

  "min_lr":MIN_LEARNING_RATE,

  "end_anneal_pc":END_LR_SMOOTHING_PERC,

  "max_mom":MOMENTUM,
 
  "min_mom":MIN_MOMENTUM,
 
  "wd":WEIGHT_DECAY,
  
  "skip_testing_epochs":0,
    
  "batches_per_epoch":50000//BATCH_SIZE,
    
  "comments":COMMENTS
}

In [0]:
loaded_tfrecs = data_pipeline.load_tfrecords(params_tune["batch_size"])

train_dataset = loaded_tfrecs["train"]

eval_dataset = loaded_tfrecs["eval"]

In [0]:
def tst_data_supplier(epoch_num):
    
    batch_size = params_tune["batch_size"]
    
    global eval_dataset
  
    len_test = 10000

    test_set = eval_dataset.map(data_aug_test).batch(batch_size).prefetch(1)
    
    return (test_set, len_test)

def trn_data_supplier(epoch_num):
    
    batch_size = params_tune["batch_size"]
  
    global train_dataset
    
    len_train = 50000

    train_set = train_dataset.map(data_aug_train).batch(batch_size).prefetch(1)
    
    return (train_set, len_train)

In [0]:
#import cProfile
model2 = ZeeDenseNet(f_filter=64,  
                     dimensions_dict= {"dimensions_to_sample":(8,8)}, 
                     gap_mode="channel_axis",
                     layers_filters={0:64, 1:128},
                    multisoft_list = [0, 1, 2],
                     roots_flag=False
                    )


64 0.6559525772546261conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
128 0.5210953257742048conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
128 0.14132361755733103conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
128 0.8856064065988761conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
256 0.6007497925514657conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
256 0.11041561005868838conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
256 0.5354799961835454conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
512 0.8186291159605041conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
512 0.4554993663837419conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
512 0.9931725773844324conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
64 0.07480373309983601conv (3, 3) (1, 1) same (2, 2) None glorot_uniform
128 0.10237239332063197conv (3, 3) (1, 1) same (2, 2) None glorot_uniform


In [0]:
x = obj.run(params_tune, trn_data_supplier, tst_data_supplier, model = model2)

epoch: 1 lr: 0.06666666666666667 train loss: 2.478 train acc:0.457 val loss: 1.885 val acc: 0.570 time:580.434
epoch: 2 lr: 0.13333333333333333 train loss: 1.490 train acc:0.695 val loss: 1.388 val acc: 0.712 time:1148.924
epoch: 3 lr: 0.2 train loss: 1.119 train acc:0.792 val loss: 1.102 val acc: 0.786 time:1717.252
epoch: 4 lr: 0.26666666666666666 train loss: 0.909 train acc:0.842 val loss: 0.885 val acc: 0.843 time:2285.412
epoch: 5 lr: 0.3333333333333333 train loss: 0.736 train acc:0.887 val loss: 0.806 val acc: 0.861 time:2853.504
epoch: 6 lr: 0.4 train loss: 0.624 train acc:0.913 val loss: 0.755 val acc: 0.875 time:3421.617
epoch: 7 lr: 0.3810526315789474 train loss: 0.571 train acc:0.927 val loss: 0.743 val acc: 0.880 time:3989.155
epoch: 8 lr: 0.36210526315789476 train loss: 0.571 train acc:0.926 val loss: 0.743 val acc: 0.880 time:4556.828
epoch: 9 lr: 0.3431578947368421 train loss: 0.568 train acc:0.929 val loss: 0.744 val acc: 0.880 time:5125.094
epoch: 10 lr: 0.324210526315

In [0]:
obj.model.summary()

In [0]:
obj.show_missclassified_images(num_images=10, tst_data_supplier=tst_data_supplier)

In [0]:
obj.plot_confusion_matrix(tst_data_supplier=tst_data_supplier)

### Grid Search

### LR Finder

Creating a separate data supplier for lr finder

In [0]:
def tst_data_supplier_lr(epoch_num):
    
    batch_size = params_tune["batch_size"]
    
    global eval_dataset
  
    len_test = 100

    test_set = eval_dataset.take(100).map(data_aug_test).batch(batch_size).prefetch(1)
    
    return (test_set, len_test)

def trn_data_supplier_lr(epoch_num):
    
    batch_size = params_tune["batch_size"]
  
    global train_dataset
    
    len_train = 1500

    train_set = train_dataset.take(len_train).map(data_aug_train).batch(batch_size).prefetch(1)
    
    return (train_set, len_train)

In [0]:
obj.lr_finder(model_fn, lr_list=[  0.004, 0.007, 0.01, 0.05, 0.09, 0.1, 0.4, 0.7, 0.9, 1], tst_data_supplier=tst_data_supplier_lr,
             trn_data_supplier=trn_data_supplier_lr)

In [0]:
model1 = ZeeDenseNet(dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16})

model2 = ZeeDenseNet(dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32})

model3 = ZeeDenseNet(dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32, 2:64})

model4 = ZeeDenseNet(f_filter=32, dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16})

model5 = ZeeDenseNet(f_filter=16, dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32, 2:64})

model6 = ZeeDenseNet(f_filter=32, dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32})

model6 = ZeeDenseNet(f_filter=32, dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32, 2:64})

params_tune_grid = {
    
    "model": [model1, model2, model3, model4, model5, model6],
    
  "epochs":[30] ,
 
  "batch_size" : [256],

  "max_lr": [0.1, 0.4, 1, 10, 0.01],

  "min_lr":[MIN_LEARNING_RATE],

  "end_anneal_pc":[END_LR_SMOOTHING_PERC],

  "max_mom":[MOMENTUM],
 
  "min_mom":[MIN_MOMENTUM],
 
  "wd":[WEIGHT_DECAY],
  
  "skip_testing_epochs":[0],
    
  "batches_per_epoch":[50000//BATCH_SIZE],
    
   "comments":["zeedense grid search"]
}


obj.grid_search(params_tune_grid, trn_data_supplier, tst_data_supplier)

##### Viewing log

In [0]:
import pandas as pd

pd.read_csv("../data/run_logger.csv").tail()

#### Numpy implementation

x_train, y_train, x_test, y_test = data_pipeline.load_saved_numpy_data(
    train_path=["../data/train/train_x.npy","../data/train/train_y.npy"],
    test_path=["../data/test/test_x.npy", "../data/test/test_y.npy"])

def tst_data_supplier(epoch_num):
    
    batch_size = params_tune["batch_size"]
    
    global eval_dataset
  
    len_test = x_test[0].shape[0]

    test_set = tf.data.Dataset.from_tensor_slices((x_test[:30,:,:,:],
                                                   y_test[:30])).batch(batch_size).prefetch(1)
    
    return (test_set, len_test)

def trn_data_supplier(epoch_num):
    
    batch_size = params_tune["batch_size"]
  
    global y_train
    
    global x_train
    
    len_test = x_train[0].shape[0]

    test_set = tf.data.Dataset.from_tensor_slices((x_train[:50,:,:,:], 
                                        y_train[:50])).batch(batch_size).prefetch(1)
    
    return (test_set, len_test)

def tst_data_supplier_lr(epoch, test_num_batches=2):
 
    batch_size = params_tune["batch_size"]
    
    global x_test
  
    global y_test
  
    sample_idx = np.random.choice(y_test.size, test_num_batches * batch_size, replace=False)

    x_test_lr = x_test[sample_idx, :, :, :]

    y_test_lr = y_test[sample_idx]
    
    len_test = x_test_lr[0].shape[0]

    test_set = tf.data.Dataset.from_tensor_slices((x_test_lr,
                                                   y_test_lr)).batch(batch_size).prefetch(1)

    return (test_set, len_test)

def trn_data_supplier_lr(epoch, train_num_batches=2):
 
    batch_size = params_tune["batch_size"]
    
    global x_train
  
    global y_train
  
    sample_idx = np.random.choice(y_train.size, train_num_batches * batch_size, replace=False)

    x_train_lr = x_train[sample_idx, :, :, :]

    y_train_lr = y_train[sample_idx]
    
    len_train = x_train_lr[0].shape[0]

    train_set = tf.data.Dataset.from_tensor_slices((x_train_lr,
                                                   y_train_lr)).batch(batch_size).prefetch(1)

    return (train_set, len_train)